In [1]:
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import StratifiedKFold

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('../')

from ract import Action, RecourseForestClassifier
from datasets import Dataset

In [2]:
DATASETS = ['c', 'f', 'b', 'p']

COST_TYPE = 'MPS'
COST_BUDGETS = [0.3]

N_ESTIMATORS = 100
MAX_DEPTH = 8
ALPHAS = {
    # 'c': [0.0, 0.02, 0.04, 0.06, 0.08, 0.1], 
    # 'f': [0.0, 0.2, 0.4, 0.6, 0.8, 1.0], 
    # 'b': [0.0, 0.02, 0.04, 0.06, 0.08, 0.1], 
    # 'p': [0.0, 0.2, 0.4, 0.6, 0.8, 1.0],
    'c': [0.0, 0.06], 
    'f': [0.0, 1.0], 
    'b': [0.0, 0.1], 
    'p': [0.0, 0.2],
}

KEYS_GROUPBY = [
    'Budget',
    'Method',
    'Alpha',
]
KEYS_MODEL = [
    'Accuracy',
    'AUC',
    'F1',
    'Time [s]',
]
KEYS_ACTION = [
    'Cost',
    'Validity',
    'Budget-Validity',
    'Valid-Cost',
    'Recourse', 
    'Sparsity', 
    'Plausibility',
]

In [3]:
def run_cv_ensemble(dataset='g', n_split=10, N=50):
    np.random.seed(0)
    
    D = Dataset(dataset=dataset)
    X, y = D.get_dataset()
    params = D.params
    
    results = {
        'Fold': [],
        'Method': [],
        'Alpha': [],
        'Budget': [],
        'Accuracy': [],
        'AUC': [],
        'F1': [],
        'Time [s]': [],
        'Cost': [],
        'Validity': [],
        'Budget-Validity': [],
        'Valid-Cost': [], 
        'Recourse': [], 
    }
    
    k = 0
    for tr, ts in StratifiedKFold(n_splits=n_split).split(X, y):
        k = k + 1
        X_tr, X_ts, y_tr, y_ts = D.X[tr], D.X[ts], D.y[tr], D.y[ts]

        
        print('- k = {}'.format(k))
        for cost_budget in COST_BUDGETS:
            action = Action(X_tr, y_target=0, cost_type=COST_TYPE, cost_budget=cost_budget, **params)

            print('\t- budget: {}'.format(cost_budget))
            for alpha in ALPHAS[dataset]:
                clf = RecourseForestClassifier(action, n_estimators=N_ESTIMATORS, max_depth=MAX_DEPTH)   
                clf.action.alpha = alpha

                start = time()
                clf = clf.fit(X_tr, y_tr)
                runtime = time() - start
                results['Fold'].append(k); results['Method'].append('Baseline' if alpha == 0.0 else 'RACT'); results['Alpha'].append(alpha); results['Budget'].append(cost_budget); 
                results['Accuracy'].append(clf.score(X_ts, y_ts)); results['AUC'].append(roc_auc_score(y_ts, clf.predict_proba(X_ts)[:, 1])); 
                results['F1'].append(f1_score(y_ts, clf.predict(X_ts))); results['Time [s]'].append(runtime); 

                X_target = X_ts[clf.predict(X_ts) != action.y_target][:N]
                n_instances = len(X_target)
                if n_instances == 0:
                    results['Cost'].append(0.0); 
                    results['Validity'].append(1.0); 
                    results['Budget-Validity'].append(1.0); 
                    results['Valid-Cost'].append(0.0); 
                    results['Recourse'].append(1.0); 
                else:
                    result = {
                        'cost': np.zeros(n_instances), 
                        'validity': np.zeros(n_instances, dtype=np.bool_), 
                        'cost-validity': np.zeros(n_instances, dtype=np.bool_), 
                        'valid-cost': [], 
                    }
                    for n, x_target in enumerate(X_target):
                        action_result = clf.explain_exact_action(x_target)
                        result['cost'][n] = action_result['cost']
                        result['validity'][n] = action_result['validity']
                        result['cost-validity'][n] = action_result['cost-validity']
                        if action_result['validity']: 
                            result['valid-cost'].append(action_result['cost'])
                    results['Cost'].append(result['cost'].mean()); 
                    results['Validity'].append(result['validity'].mean()); 
                    results['Budget-Validity'].append(result['cost-validity'].mean()); 
                    results['Valid-Cost'].append(np.mean(result['valid-cost'])); 
                    results['Recourse'].append(1 - (~result['cost-validity']).sum() / X_ts.shape[0]); 

                print('\t\t- Method: {} (alp.: {}) | AUC: {:.4} | Val.: {:.4} | Rec.: {:.4}'.format(results['Method'][-1], alpha, results['AUC'][-1], results['Budget-Validity'][-1], results['Recourse'][-1]))

                if alpha == 0.0:
                    clf = RecourseForestClassifier(action, n_estimators=N_ESTIMATORS, max_depth=MAX_DEPTH, feature_masking=True)

                    start = time()
                    clf = clf.fit(X_tr, y_tr)
                    runtime = time() - start
                    results['Fold'].append(k); results['Method'].append('Baseline (OAF)'); results['Alpha'].append(alpha); results['Budget'].append(cost_budget); 
                    results['Accuracy'].append(clf.score(X_ts, y_ts)); results['AUC'].append(roc_auc_score(y_ts, clf.predict_proba(X_ts)[:, 1])); 
                    results['F1'].append(f1_score(y_ts, clf.predict(X_ts))); results['Time [s]'].append(runtime); 

                    X_target = X_ts[clf.predict(X_ts) != action.y_target][:N]
                    n_instances = len(X_target)
                    if n_instances == 0:
                        results['Cost'].append(0.0); 
                        results['Validity'].append(1.0); 
                        results['Budget-Validity'].append(1.0); 
                        results['Valid-Cost'].append(0.0); 
                        results['Recourse'].append(1.0); 
                    else:
                        result = {
                            'cost': np.zeros(n_instances), 
                            'validity': np.zeros(n_instances, dtype=np.bool_), 
                            'cost-validity': np.zeros(n_instances, dtype=np.bool_), 
                            'valid-cost': [], 
                        }
                        for n, x_target in enumerate(X_target):
                            action_result = clf.explain_exact_action(x_target)
                            result['cost'][n] = action_result['cost']
                            result['validity'][n] = action_result['validity']
                            result['cost-validity'][n] = action_result['cost-validity']
                            if action_result['validity']: 
                                result['valid-cost'].append(action_result['cost'])
                        results['Cost'].append(result['cost'].mean()); 
                        results['Validity'].append(result['validity'].mean()); 
                        results['Budget-Validity'].append(result['cost-validity'].mean()); 
                        results['Valid-Cost'].append(np.mean(result['valid-cost'])); 
                        results['Recourse'].append(1 - (~result['cost-validity']).sum() / X_ts.shape[0]); 

                    print('\t\t- Method: {} | AUC: {:.4} | Val.: {:.4} | Rec.: {:.4}'.format(results['Method'][-1], results['AUC'][-1], results['Budget-Validity'][-1], results['Recourse'][-1]))
        
    
    results = pd.DataFrame(results)
    results.to_csv('../res/appendix/milo/{}_forest.csv'.format(dataset), index=False)
    

In [4]:
from datasets import DATASET_FULLNAMES

for dataset in DATASETS: 
    print('# {}'.format(DATASET_FULLNAMES[dataset]))
    run_cv_ensemble(dataset)
    print()

# COMPAS
- k = 1
	- budget: 0.3
		- Method: Baseline (alp.: 0.0) | AUC: 0.7276 | Val.: 0.64 | Rec.: 0.9708
		- Method: Baseline (OAF) | AUC: 0.7341 | Val.: 0.7 | Rec.: 0.9757
		- Method: RACT (alp.: 0.06) | AUC: 0.7258 | Val.: 0.9 | Rec.: 0.9919
- k = 2
	- budget: 0.3
		- Method: Baseline (alp.: 0.0) | AUC: 0.7384 | Val.: 0.72 | Rec.: 0.9773
		- Method: Baseline (OAF) | AUC: 0.7309 | Val.: 0.74 | Rec.: 0.9789
		- Method: RACT (alp.: 0.06) | AUC: 0.7373 | Val.: 0.82 | Rec.: 0.9854
- k = 3
	- budget: 0.3
		- Method: Baseline (alp.: 0.0) | AUC: 0.6928 | Val.: 0.6 | Rec.: 0.9676
		- Method: Baseline (OAF) | AUC: 0.6863 | Val.: 0.8 | Rec.: 0.9838
		- Method: RACT (alp.: 0.06) | AUC: 0.692 | Val.: 0.86 | Rec.: 0.9887
- k = 4
	- budget: 0.3
		- Method: Baseline (alp.: 0.0) | AUC: 0.7273 | Val.: 0.82 | Rec.: 0.9854
		- Method: Baseline (OAF) | AUC: 0.722 | Val.: 0.88 | Rec.: 0.9903
		- Method: RACT (alp.: 0.06) | AUC: 0.7282 | Val.: 1.0 | Rec.: 1.0
- k = 5
	- budget: 0.3
		- Method: Baseline (